In [1]:
import re
import os
import json
import pandas as pd
import ast
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:

def merge_csv_files_from_folder(folder_path):
    # Lista para almacenar los DataFrames
    df_list = []

    # Recorrer todos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Obtener la ruta completa del archivo
            file_path = os.path.join(folder_path, filename)
            
            # Leer el archivo CSV y agregarlo a la lista de DataFrames
            df = pd.read_csv(file_path)
            df_list.append(df)

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("No se encontraron archivos CSV en la carpeta.")
        return pd.DataFrame()  

data = merge_csv_files_from_folder("data_processed/csv/")
data.drop(columns="car_id", inplace=True)
colnames = data.columns.to_list()
for col in colnames: print(col)

C:\Users\lingf\AppData\Local\Temp\ipykernel_8160\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_8160\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_8160\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_8160\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_8160\2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option 

price
km
year
color
cubicCapacity
brand
model
version
fuelType
bodyTypeId
transmissionTypeId
doors
seatingCapacity
warranty_months
province
environmentalLabel
co2Emissions
co2EmissionsGramsPerKm
trunkCapacityInLiters
maxSpeed
acceleration
hasDetails
jato_classification
traction
brakes
front_suspension
rear_suspension
power_cv
power_kw
rpm_max_power
max_torque_nm
rpm_max_torque
motor_description
transmission_description
speakers
trip_computer
remote_audio_control_on_steering_wheel
dvd_navigation
antenna
abs
electronic_traction_control
parking_sensors
airbag
stability_control
curve_braking_control
isofix_system
start_stop_automatic
cubicCapacity_seat_folded
Acabado Interior_Apoyabrazos
Acabado Interior_Asiento
Acabado Interior_Asientos
Acabado Interior_Dos
Acabado Interior_Pedales
Acabado Interior_Reloj
Acabado Interior_Reposacabezas
Acabado Interior_Tres
Acabado Interior_Uno
Confort_Aire
Confort_Apertura
Confort_Aviso
Confort_Banda
Confort_Calefacción
Confort_Cierre
Confort_Compartiment

In [3]:
for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].map({"True": True, "False": False})

C:\Users\lingf\AppData\Local\Temp\ipykernel_8160\2833469390.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
165432    NaN
165433    NaN
165434    NaN
165435    NaN
165436    NaN
Name: cubicCapacity_seat_folded, Length: 165437, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].map({"True": True, "False": False})


In [4]:
def find_dirty_columns(prefix): return [col for col in data.columns if prefix in col]

# Clean Dirty Columns
Lets Clean some dirty data

## Clean Confort


In [5]:
confort_columns = find_dirty_columns("Confort")
for x in confort_columns: print(x)

Confort_Aire
Confort_Apertura
Confort_Aviso
Confort_Banda
Confort_Calefacción
Confort_Cierre
Confort_Compartimentos
Confort_Control
Confort_Dirección
Confort_Dos
Confort_Elevalunas
Confort_Faros
Confort_Indicador
Confort_Inmovilizador
Confort_Inmovilizador

Confort_Intermitentes
Confort_Lavafaros
Confort_Lavafaros

Confort_Limpiaparabrisas
Confort_Llave
Confort_Luces
Confort_Luz
Confort_Memoria
Confort_Protección
Confort_Regulación
Confort_Sistema
Confort_Tapa
Confort_Tarjeta
Confort_Techo
Confort_Testigo
Confort_Una
Confort_Volante


## Clean Acabado Interior 

In [6]:
acabado_interior_columns = find_dirty_columns("Acabado Interior")

In [7]:
acabado_interior_columns.remove("Acabado Interior_Asientos")
data.drop(columns="Acabado Interior_Asientos", inplace=True)

for x in acabado_interior_columns: print(x)
data[acabado_interior_columns]

Acabado Interior_Apoyabrazos
Acabado Interior_Asiento
Acabado Interior_Dos
Acabado Interior_Pedales
Acabado Interior_Reloj
Acabado Interior_Reposacabezas
Acabado Interior_Tres
Acabado Interior_Uno


,Acabado Interior_Apoyabrazos,Acabado Interior_Asiento,Acabado Interior_Dos,Acabado Interior_Pedales,Acabado Interior_Reloj,Acabado Interior_Reposacabezas,Acabado Interior_Tres,Acabado Interior_Uno
0,['Apoyabrazos central delantero con caja integ...,['Asiento delantero del conductor y acompañant...,['Dos reposacabezas en asientos delanteros'],no tiene,['Reloj'],no tiene,no tiene,no tiene
1,['Apoyabrazos central delantero'],['Asiento delantero del conductor y acompañant...,['Dos reposacabezas activos en asientos delant...,no tiene,no tiene,no tiene,no tiene,no tiene
2,['Apoyabrazos central delantero'],['Asiento delantero del conductor deportivo co...,['Dos reposacabezas en asientos delanteros'],no tiene,no tiene,no tiene,no tiene,no tiene
3,['Apoyabrazos central delantero'],"['Asiento delantero del conductor individual, ...",['Dos reposacabezas en asientos delanteros y a...,no tiene,no tiene,no tiene,no tiene,no tiene
4,"['Apoyabrazos central delantero', 'Apoyabrazos...",['Asiento delantero del conductor y acompañant...,['Dos reposacabezas en asientos delanteros aju...,no tiene,no tiene,no tiene,no tiene,no tiene
...,...,...,...,...,...,...,...,...
165432,"['Apoyabrazos central delantero', 'Apoyabrazos...",['Asiento delantero del conductor individual c...,['Dos reposacabezas en asientos delanteros aju...,no tiene,no tiene,no tiene,no tiene,no tiene
165433,no tiene,['Asiento delantero del conductor y acompañant...,['Dos reposacabezas en asientos delanteros aju...,no tiene,no tiene,no tiene,no tiene,no tiene
165434,"['Apoyabrazos central delantero', 'Apoyabrazos...",['Asiento delantero del conductor individual c...,['Dos reposacabezas en asientos delanteros aju...,no tiene,no tiene,no tiene,no tiene,no tiene
165435,"['Apoyabrazos central delantero', 'Apoyabrazos...",['Asiento delantero del conductor y acompañant...,['Dos reposacabezas en asientos delanteros aju...,no tiene,no tiene,no tiene,no tiene,no tiene


In [8]:
def check_unique_values(column):
    ret = sorted(set(data[column].to_list()))
    print(f"Column: {column}")
    for x in ret: print(f'{x}: {data[column].to_list().count(x)}')
    print(f'NA: {data[column].to_list().count(None)}')

check_unique_values("Acabado Interior_Tres")

Column: Acabado Interior_Tres
['Tres reposacabezas en F ajustables en altura']: 121
['Tres reposacabezas en F ajustables en altura, cuatro reposacabezas en R ajustables en altura']: 2
['Tres reposacabezas en F ajustables en altura, cuatro reposacabezas en R']: 1
['Tres reposacabezas en F ajustables en altura, dos reposacabezas en R ajustables en altura']: 1
['Tres reposacabezas en F ajustables en altura, dos reposacabezas en R ajustables en altura, tres reposacabezas en la tercera fila de asientos ajustables en altura']: 1
['Tres reposacabezas en F ajustables en altura, tres reposacabezas en R, tres reposacabezas en la tercera fila de asientos']: 1
['Tres reposacabezas en F y R ajustables en altura']: 34
['Tres reposacabezas en F']: 11
['Tres reposacabezas en F, R y la tercera fila de asientos ajustables en altura']: 7
['Tres reposacabezas en F, cuatro reposacabezas en R']: 5
['Tres reposacabezas en F, tres reposacabezas en R ajustables en altura']: 2
['Tres reposacabezas en F, tres re

In [9]:
data["sports_pedals"] = data["Acabado Interior_Pedales"].map({
    "['Pedales deportivos']": True, 
    'no tiene': False
})
data["interior_clock"] = data["Acabado Interior_Reloj"].map({
    "['Reloj analógico']": 'analogico', 
    "['Reloj digital']": 'digital',
    "['Reloj']": 'regular',
    "no tiene": 'no tiene'
})

#check_unique_values("sports_pedals")
#check_unique_values("interior_clock")


In [10]:

def process_arm_suppport_front(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('delantero' in text): return ("no tiene")
    text = [word for word in text.split() if word not in stop_words]
    if '#' in text: return " ".join( text[ (text.index('delantero')+1):text.index('#')] )
    else: return " ".join( text[(text.index('delantero')+1):len(text)] ) 

def process_arm_suppport_back(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('trasero' in text): return ("no tiene")

    text = [word for word in text.split() if word not in stop_words]

    return " ".join( text[(text.index('trasero')+1):len(text)] ) 

data["arm_suppport_front"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_front(eval(x)) )
data["arm_suppport_back"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_back(eval(x)) )

#check_unique_values("arm_suppport_front")
#check_unique_values("arm_suppport_back")

In [11]:
def process_seat_description(phrases: list) -> str: return ". ".join(phrases)

data["seat_description"] = data["Acabado Interior_Asiento"].map( lambda x: x if x == 'no tiene' else process_seat_description(eval(x)) )

In [12]:
data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )

data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].map(ast.literal_eval)
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].map(ast.literal_eval)
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].map(ast.literal_eval)
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].map(ast.literal_eval)

In [18]:
def remove_duplicates(lista, elemento): return [x for x in lista if x != elemento]
data['head_supporter'] = data.apply(
    lambda row: ". ".join(
        remove_duplicates(row["Acabado Interior_Uno"] + row["Acabado Interior_Dos"] + row["Acabado Interior_Tres"] + row["Acabado Interior_Reposacabezas"], 'no tiene')
    ), 
    axis=1)
check_unique_values("head_supporter")

for col in acabado_interior_columns: data.drop(columns= col, inplace=True)

Column: head_supporter
: 72
Dos reposacabezas activos en F ajustables en altura: 36
Dos reposacabezas activos en F ajustables en altura y eléctricamente, tres reposacabezas en R: 15
Dos reposacabezas activos en F ajustables en altura y eléctricamente, tres reposacabezas en R ajustables en altura: 6
Dos reposacabezas activos en F ajustables en altura, dos reposacabezas en R: 161
Dos reposacabezas activos en F ajustables en altura, dos reposacabezas en R ajustables en altura: 320
Dos reposacabezas activos en F ajustables en altura, dos reposacabezas en R ajustables en altura, tres reposacabezas en la tercera fila de asientos ajustables en altura: 1
Dos reposacabezas activos en F ajustables en altura, dos reposacabezas en R, tres reposacabezas en la tercera fila de asientos ajustables en altura: 5
Dos reposacabezas activos en F ajustables en altura, tres reposacabezas activos en R: 2
Dos reposacabezas activos en F ajustables en altura, tres reposacabezas activos en R ajustables en altura:

## Clean Acabado Exterior

In [13]:
acabado_exterior_columns = find_dirty_columns("Acabado Exterior")
for x in acabado_exterior_columns: print(x)
data[acabado_exterior_columns]

Acabado Exterior_Alerón
Acabado Exterior_Cristales
Acabado Exterior_Defensa
Acabado Exterior_Faldones
Acabado Exterior_Guardabarros
Acabado Exterior_Llantas
Acabado Exterior_Maletero
Acabado Exterior_Molduras
Acabado Exterior_Neumáticos
Acabado Exterior_Pasos
Acabado Exterior_Pintura
Acabado Exterior_Puerta
Acabado Exterior_Retrovisor
Acabado Exterior_Rueda


,Acabado Exterior_Alerón,Acabado Exterior_Cristales,Acabado Exterior_Defensa,Acabado Exterior_Faldones,Acabado Exterior_Guardabarros,Acabado Exterior_Llantas,Acabado Exterior_Maletero,Acabado Exterior_Molduras,Acabado Exterior_Neumáticos,Acabado Exterior_Pasos,Acabado Exterior_Pintura,Acabado Exterior_Puerta,Acabado Exterior_Retrovisor,Acabado Exterior_Rueda
0,no tiene,no tiene,no tiene,no tiene,no tiene,['Llantas delanteras y traseras de aleación li...,no tiene,no tiene,['Neumáticos delanteros y traseros 205 mm de a...,no tiene,['Pintura solida'],['Puerta conductor y pasajero con visagras del...,['Retrovisor exterior lado conductor y acompañ...,no tiene
1,no tiene,['Cristales tintados'],no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aleación li...,no tiene,no tiene,['Neumáticos delanteros y traseros de 18 pulga...,no tiene,['Pintura solida'],['Puerta conductor y pasajero con bisagras del...,['Retrovisor exterior del conductor y acompaña...,no tiene
2,['Alerón en el maletero/parte inferior del por...,['Cristales tintados'],no tiene,no tiene,no tiene,['Llantas delanteras en aleación ligera de 20 ...,no tiene,no tiene,['Neumáticos delanteros de 20 pulgadas de diam...,no tiene,['Pintura metalizada'],['Puerta conductor y pasajero con bisagras del...,['Retrovisor exterior del conductor pintado co...,no tiene
3,['Alerón en el techo/parte superior del portón'],no tiene,no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aluminio de...,no tiene,no tiene,['Neumáticos delanteros y traseros de 17 pulga...,no tiene,['Pintura metalizada'],"['Puerta conductor con bisagras delanteras, pu...",['Retrovisor exterior del conductor y acompaña...,no tiene
4,no tiene,no tiene,no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aluminio de...,no tiene,no tiene,['Neumáticos delanteros y traseros de 18 pulga...,no tiene,['Pintura solida'],"['Puerta conductor, trasera (lado conductor), ...",['Retrovisor exterior del conductor en color c...,no tiene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165432,['Alerón en el techo/parte superior del portón'],['Cristales tintados'],no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aleación li...,no tiene,no tiene,['Neumáticos delanteros y traseros de 16 pulga...,no tiene,['Pintura metalizada'],"['Puerta conductor, trasera (lado conductor), ...",['Retrovisor exterior del conductor y acompaña...,['Rueda de repuesto con llanta en acero de men...
165433,['Alerón en el techo/parte superior del portón'],no tiene,no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aluminio de...,no tiene,no tiene,['Neumáticos delanteros y traseros de 16 pulga...,no tiene,['Pintura bicolor solida'],"['Puerta conductor, trasera (lado conductor), ...",['Retrovisor exterior del conductor y acompaña...,no tiene
165434,['Alerón en el techo/parte superior del portón'],no tiene,no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aleación li...,no tiene,no tiene,['Neumáticos delanteros y traseros de 17 pulga...,no tiene,['Pintura solida'],"['Puerta conductor, trasera (lado conductor), ...",['Retrovisor exterior del conductor y acompaña...,no tiene
165435,['Alerón en el techo/parte superior del portón'],no tiene,no tiene,no tiene,no tiene,['Llantas delanteras y traseras en aluminio de...,no tiene,no tiene,['Neumáticos delanteros y traseros de 18 pulga...,no tiene,['Pintura solida'],"['Puerta conductor, trasera (lado conductor), ...",['Retrovisor exterior del conductor pintado co...,no tiene


## Electronic Features

In [14]:
print(data["fuelType"])
check_unique_values("fuelType")


0                   Gasolina
1                   Gasolina
2                   Gasolina
3                  Eléctrico
4                    Híbrido
                 ...        
165432                Diésel
165433              Gasolina
165434               Híbrido
165435    Híbrido enchufable
165436              Gasolina
Name: fuelType, Length: 165437, dtype: object


TypeError: '<' not supported between instances of 'float' and 'str'